1. Import modules and functions

In [27]:
import os
import sys
from pathlib import Path
import glob
import pandas as pd
from datasets import Dataset, load_from_disk, concatenate_datasets, DatasetInfo, DatasetDict
from PIL import Image

from src.extract_create import *


2. Extract text regions and crop images

In [ ]:
input_folder = Path("data/raw/")
output_folder = Path("data/processed")
extract_and_create(input_folder, output_folder)

# Create the combined dataset and upload it to HuggingFace

In [25]:
text_dir = "data/processed/texts"
image_dir = "data/processed/images"

dataframes = [pd.read_csv(x) for x in glob.glob(f"{text_dir}/*.csv")]


In [ ]:
# Combine datasets from each dataframe
datasets = [process_dataframe(df,image_dir) for df in dataframes]
combined_dataset = concatenate_datasets(datasets)

Save the combined dataset

In [37]:
combined_dataset.save_to_disk("data/processed/combined_dataset_p40")

Saving the dataset (0/1 shards):   0%|          | 0/661 [00:00<?, ? examples/s]

## Spliting into train and validation set

In [ ]:
train_test_split = combined_dataset.train_test_split(test_size=0.2)

split_dataset = DatasetDict({
    'train': train_test_split['train'],
    'validation': train_test_split['test']
})

In [ ]:
### Saving to HF hub

In [ ]:
with open("tokens/token") as f:
    token = f.read()

In [ ]:
split_dataset.push_to_hub("mevsg/Gongguan-OCR-v1", token=token)